In [1]:
# 필요한 라이브러리 다시 불러오기
import numpy as np
import pandas as pd

panel_df = pd.read_excel('./data/태양광 패널 정보 최종.xlsx')
panel_df2 = panel_df[['제품명', '가격', '최대 출력', '효율', '가로', '세로','가격당 최대출력(*100K)', '보증기간', '중국산 여부']]
panel_df



,제품명,가격,최대 출력,효율,가격당 최대출력(*100K),정격전압,개방 전압,가로,세로,두께,면적,무게,휴대용 파워뱅크,결정구조,커넥터,프레임,제조사,중국산 여부,보증기간,기타
0,한화큐셀 태양광 패널 모듈 310W,293420,310,18.90,1056.506032,32.99,39.87,1670,1000,32,1.67000,18.5,NaN,단결정,NaN,NaN,한화,NaN,12,NaN
1,MD홍 플렉시블 태양광패널 300W,268000,300,18.44,1119.402985,36.00,44.57,1640,992,35,1.62688,17.5,NaN,다결정,NaN,NaN,MD홍,NaN,NaN,NaN
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,17.60,1101.694915,37.00,34.90,1480,998,35,1.47704,18.7,NaN,단결정,NaN,알루미늄 합금,현대중공업,NaN,12,NaN
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,NaN,1705.514497,32.20,38.80,1640,995,35,1.63180,NaN,NaN,단결정,MC4,NaN,seirifo,O,X,NaN
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,NaN,1700.680272,35.80,39.50,1680,1000,35,1.68000,NaN,NaN,다결정,MC4,NaN,seirifo,O,X,NaN
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,NaN,1695.671575,32.80,39.50,1760,1050,35,1.84800,NaN,NaN,다결정,MC4,NaN,seirifo,O,X,NaN
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,NaN,2012.072435,41.70,49.50,2100,1050,35,2.20500,NaN,NaN,다결정,MC4,NaN,seirifo,O,X,NaN


In [2]:
panel_df2

,제품명,가격,최대 출력,효율,가로,세로,가격당 최대출력(*100K),보증기간,중국산 여부
0,한화큐셀 태양광 패널 모듈 310W,293420,310,18.90,1670,1000,1056.506032,12,NaN
1,MD홍 플렉시블 태양광패널 300W,268000,300,18.44,1640,992,1119.402985,NaN,NaN
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,17.60,1480,998,1101.694915,12,NaN
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,NaN,1640,995,1705.514497,X,O
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,NaN,1680,1000,1700.680272,X,O
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,NaN,1760,1050,1695.671575,X,O
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,NaN,2100,1050,2012.072435,X,O


In [3]:
# 테스트용으로 신당동 1729-9  건물정보를 입력하였음
user_dic = {
    '부지가로' : 10.4,              # 부지가로길이 단위는 m
    '부지세로' : 10.4,              # 부지세로길이 단위는 m
    '단위면적당 일사량' : 919	,     # 단위면적당 일사량은 1m^2의 면적당 일사량을 의미. 단위는 kWh/년
    '총예산' : 100 ,               # 총예산은 만원 단위
    '1kW당 가격' : 165             # 1kW당 가격. (입지에 따라 달라짐). 단위는 원
}

# 부지에 패널당 최대 패널 개수 구하는 함수 fill_problem의 버그 고치고 패널당 손익분기점 계산하고 가장 빠른거를 따로 출력해줘야함

### 기후내용도 들어가야하는데 일단 나중에...
### 그다음은 인버터도 고려해야하지만 이것도 나중에..

In [4]:
# panel_df2의 각각 패널에 설치 가능 개수를 세어주는 함수이다

def fill_problem(panel_df2, user_dic):
    # 입력 값
    module_width = panel_df2['가로']  # 모듈 가로, mm
    module_height = panel_df2['세로']  # 모듈 세로, mm
    site_width = user_dic['부지가로']*1000  # 설치할 부지 가로, mm (1m = 1,000mm)
    site_hieght = user_dic['부지세로']*1000 # 설치할 부지 세로, mm (1m = 1,000mm)



    # 모듈 사이에 거리 띄우기 (마진)     
    # 그림자, 공기 흐름, 온도, 사람 지나다닐 공간 등을 고려하여 15cm를 띄웠음
    module_total_width = module_width + 150
    module_total_height = module_height + 150

    # 모듈을 부지에 채우기 (부지가 직사각형이라고 가정하고 채우는 것이므로 오차가 있을 수 있음)
    module_width_count = site_width // module_total_width
    module_height_count = site_hieght // module_total_height
    count = module_width_count * module_height_count


    # 모듈의 가로 세로 바꾸고(옆으로 눕혀서) 부지에 채우기
    module_width_count = site_width // module_total_width
    module_height_count = site_hieght // module_total_height
    count2 = module_width_count * module_height_count
    
    module_max_count=[]
    # 둘 중 많이 카운트 된 것을 모듈 최대 개수로 계산
    for c,c2 in zip(count, count2):
        if(c > c2):
            module_max_count.append(c)
        else :
            module_max_count.append(c2)

    return module_max_count

In [5]:
# panel_df2의 각각 패널에 설치 가능 개수를 세어준 것을 열추가 해준다.

module_max_count = fill_problem(panel_df2, user_dic)
result_df= pd.DataFrame()
result_df['제품명'] = panel_df2['제품명']
result_df['가격'] = panel_df2['가격']
result_df['최대 출력'] = panel_df2['최대 출력']
result_df['설치개수(max)'] = module_max_count
result_df['비용(max)'] = result_df['설치개수(max)'] * result_df['가격']
result_df['총 발전용량(max)'] = result_df['설치개수(max)'] * result_df['최대 출력']
result_df['가격당 최대출력'] = panel_df2['가격당 최대출력(*100K)']
result_df['중국산 여부'] = panel_df2['중국산 여부']
result_df

,제품명,가격,최대 출력,설치개수(max),비용(max),총 발전용량(max),가격당 최대출력,중국산 여부
0,한화큐셀 태양광 패널 모듈 310W,293420,310,45.0,13203900.0,13950.0,1056.506032,NaN
1,MD홍 플렉시블 태양광패널 300W,268000,300,45.0,12060000.0,13500.0,1119.402985,NaN
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,54.0,12744000.0,14040.0,1101.694915,NaN
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,45.0,7915500.0,13500.0,1705.514497,O
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,45.0,9261000.0,15750.0,1700.680272,O
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,40.0,8964000.0,15200.0,1695.671575,O
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,32.0,7952000.0,16000.0,2012.072435,O


<h4 style="color:red"> 사용자가 입력한 총 발전용량에 맞춰서 설치개수, 비용, 총 발전용량이 계산하는 함수 </h4>

In [6]:
# 검색에 사용자가 총발전용량을 5000(5k) 입력하면
# 총 발전용량을 5000(5k)에 최대한 근접하게 맞춰서 다시 분석해서 result_df에 넣는다
test_capacity = 5000  

def filter_with_capacity(input_total_capacity):
    # 각 행을 순회하면서 새로운 값 계산하여 추가
    for index, row in result_df.iterrows():
        # 예: A와 B 값을 더한 값을 새로운 열에 추가
        panel_count = int(input_total_capacity / row['최대 출력'])
        
        # if(panel_count > row['설치개수(max)']):
        #     panel_count = row['설치개수(max)']
        
        # 해당 행의 새로운 열에 값 할당
        result_df.at[index, '설치개수'] = panel_count
        result_df.at[index, '비용'] = panel_count * row['가격']
        result_df.at[index, '총 발전용량'] = panel_count * row['최대 출력']


result_df

,제품명,가격,최대 출력,설치개수(max),비용(max),총 발전용량(max),가격당 최대출력,중국산 여부
0,한화큐셀 태양광 패널 모듈 310W,293420,310,45.0,13203900.0,13950.0,1056.506032,NaN
1,MD홍 플렉시블 태양광패널 300W,268000,300,45.0,12060000.0,13500.0,1119.402985,NaN
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,54.0,12744000.0,14040.0,1101.694915,NaN
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,45.0,7915500.0,13500.0,1705.514497,O
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,45.0,9261000.0,15750.0,1700.680272,O
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,40.0,8964000.0,15200.0,1695.671575,O
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,32.0,7952000.0,16000.0,2012.072435,O


<h4 style="color:red"> 사용자가 입력한 예산에 맞춰서 설치개수, 비용, 총 발전용량이 계산하는 함수 </h4>

In [7]:
# 사용자가 입력한 총예산. 단위는 만원
test_budget = 300 

def filter_with_budget(input_total_budget):
    # 각 행을 순회하면서 새로운 값 계산하여 추가
    for index, row in result_df.iterrows():
        # 예: A와 B 값을 더한 값을 새로운 열에 추가
        panel_count = int(input_total_budget * 10000 / row['가격'])
        
        # 해당 행의 새로운 열에 값 할당
        result_df.at[index, '설치개수'] = panel_count
        result_df.at[index, '비용'] = panel_count * row['가격']
        result_df.at[index,'총 발전용량'] = panel_count * row['최대 출력']

result_df

,제품명,가격,최대 출력,설치개수(max),비용(max),총 발전용량(max),가격당 최대출력,중국산 여부
0,한화큐셀 태양광 패널 모듈 310W,293420,310,45.0,13203900.0,13950.0,1056.506032,NaN
1,MD홍 플렉시블 태양광패널 300W,268000,300,45.0,12060000.0,13500.0,1119.402985,NaN
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,54.0,12744000.0,14040.0,1101.694915,NaN
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,45.0,7915500.0,13500.0,1705.514497,O
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,45.0,9261000.0,15750.0,1700.680272,O
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,40.0,8964000.0,15200.0,1695.671575,O
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,32.0,7952000.0,16000.0,2012.072435,O


<h2 style="background-color:#C0FFFF; color:blue"> 이제부터 손익분기점을 게산하기 위해 1년당 이익을 계산한다.</h2>
<h2>일단, 총 발전용량을 5000W(테스트)이하로 필터 설정하고 계산한다</h2>

In [8]:
test_capacity = 5000
filter_with_capacity(test_capacity)

<h2 style="background-color:#C0FFFF; color:black">손익분기점이 가장 빠른 순서대로 정렬해서 보여줌</h2>

In [9]:
import numpy as np

# kW단위이므로 총 발전용량에서 1000을 나눠준다
result_df['연간 전기 생산량(kWh/년)'] =  (result_df['총 발전용량'] /1000)* user_dic['단위면적당 일사량']
result_df['매출액(원/년)'] = result_df['연간 전기 생산량(kWh/년)'] * user_dic['1kW당 가격']
result_df['손익분기점_개월'] = np.floor(result_df['비용'] / (result_df['매출액(원/년)'] / 12)).astype(int)
# result_df

result_df_sorted = result_df.sort_values(by='손익분기점_개월', ascending=True)
result_df_sorted

,제품명,가격,최대 출력,설치개수(max),비용(max),총 발전용량(max),가격당 최대출력,중국산 여부,설치개수,비용,총 발전용량,연간 전기 생산량(kWh/년),매출액(원/년),손익분기점_개월
6,태양광 패널 500W 태양열판 가정용 플랙시블 가로등,248500,500,32.0,7952000.0,16000.0,2012.072435,O,10.0,2485000.0,5000.0,4595.00,758175.0,39
3,태양광 패널 300W 태양열판 가정용 플랙시블 가로등,175900,300,45.0,7915500.0,13500.0,1705.514497,O,16.0,2814400.0,4800.0,4411.20,727848.0,46
4,태양광 패널 350W 태양열판 가정용 플랙시블 가로등,205800,350,45.0,9261000.0,15750.0,1700.680272,O,14.0,2881200.0,4900.0,4503.10,743011.5,46
5,태양광 패널 380W 태양열판 가정용 플랙시블 가로등,224100,380,40.0,8964000.0,15200.0,1695.671575,O,13.0,2913300.0,4940.0,4539.86,749076.9,46
1,MD홍 플렉시블 태양광패널 300W,268000,300,45.0,12060000.0,13500.0,1119.402985,NaN,16.0,4288000.0,4800.0,4411.20,727848.0,70
2,현대에너지 태양광 패널 모듈 260W HiS-S260RF,236000,260,54.0,12744000.0,14040.0,1101.694915,NaN,19.0,4484000.0,4940.0,4539.86,749076.9,71
0,한화큐셀 태양광 패널 모듈 310W,293420,310,45.0,13203900.0,13950.0,1056.506032,NaN,16.0,4694720.0,4960.0,4558.24,752109.6,74
